In [5]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dataset imdb_reviews downloaded and prepared to /Users/nasimulalahi/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [10]:
imdb_train, imdb_test = imdb['train'], imdb['test']

In [17]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# this resembles a key:value pair where s = sentence = key, l = label = value
for s,l in imdb_train:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())
    
for s,l in imdb_test:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [18]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [19]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [26]:
# transform word:index to index:word
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# prints out the sentences as learnt by the tokenizer with padding
print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [22]:
# deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compilation of the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# prints out the model summary
model.summary()

# The results of the embedding will be a 2D array with the length of
# the sentence and the embedding dimension for example 16 as its size.

# tf.keras.layers.GlobalAveragePooling1D() can be used instead of tf.keras.layers.flatten()
# which averages across the vector to flatten it out. the outcome would be much simpler and
# training time would be less but the accuracy score would fall down

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [23]:
# training the model
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 5ms/step - loss: 0.6045 - accuracy: 0.6429 - val_loss: 0.3438 - val_accuracy: 0.8528
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2445 - accuracy: 0.9066 - val_loss: 0.3660 - val_accuracy: 0.8398
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1030 - accuracy: 0.9742 - val_loss: 0.4497 - val_accuracy: 0.8301
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0335 - accuracy: 0.9947 - val_loss: 0.5234 - val_accuracy: 0.8271
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0122 - accuracy: 0.9985 - val_loss: 0.5831 - val_accuracy: 0.8256
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0071 - accuracy: 0.9989 - val_loss: 0.6461 - val_accuracy: 0.8243
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0041 - accuracy: 0.9993 - val_loss: 0.7096 - val_accuracy: 0.8239
Epoch 

In [24]:
# get the shape of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [28]:
# this part of code should be further monitored #
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [30]:
# way to download files from google colaboratory
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [31]:
# way to download files from jupyter notebook
from IPython.display import FileLink

display(FileLink('vecs.tsv'))
display(FileLink('meta.tsv'))

/Users/nasimulalahi/Desktop/Natural Language Processing in TensorFlow/vecs.tsv

/Users/nasimulalahi/Desktop/Natural Language Processing in TensorFlow/meta.tsv

In [32]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
